In [1]:
import pandas as pd
import numpy as np
import json
import requests
from time import sleep
import pprint

# Get Data from API using query - 2019

In [2]:
data = []

#query = "q=&deepsearch=on&declaration_year=2019&doc_type=Після+звільнення"
query = "q=Народний+депутат+України&declaration_year=2019&doc_type=Щорічна"


print("Fetching page #%s" % 1)

r = requests.get("http://declarations.com.ua/search?"+ query + "&format=opendata").json()
data += r["results"]["object_list"]

Fetching page #1


In [3]:
for page in range(2, r["results"]["paginator"]["num_pages"] + 1):
    sleep(0.5)
    print("Fetching page #%s" % page)

    subr = requests.get(
        "http://declarations.com.ua/search?"+ query + "&format=opendata&page=%s" % page).json()
    data += subr["results"]["object_list"]

print("Declarations exported %s" % len(data))

with open("NarDep2019.json", "w") as fp:
    json.dump(data, fp)

Fetching page #2
Fetching page #3
Fetching page #4
Fetching page #5
Fetching page #6
Fetching page #7
Fetching page #8
Fetching page #9
Fetching page #10
Fetching page #11
Fetching page #12
Fetching page #13
Fetching page #14
Fetching page #15
Fetching page #16
Fetching page #17
Fetching page #18
Fetching page #19
Fetching page #20
Fetching page #21
Fetching page #22
Fetching page #23
Fetching page #24
Fetching page #25
Fetching page #26
Fetching page #27
Declarations exported 787


In [4]:
print(len(data))

787


In [5]:
declaration = data[0] #json.loads()
declaration.keys()

dict_keys(['guid', 'infocard', 'raw_source', 'unified_source', 'related_entities'])

In [6]:
pprint.pprint(declaration)

{'guid': 'nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
 'infocard': {'created_date': '2020-01-28T12:40:14',
              'declaration_year': 2019,
              'document_type': 'Щорічна',
              'first_name': 'Олександр',
              'id': 'nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
              'is_corrected': False,
              'last_name': 'Сухов',
              'office': 'народний депутат України ',
              'patronymic': 'Сергійович',
              'position': 'народний депутат України ',
              'source': 'NACP',
              'url': 'https://declarations.com.ua/declaration/nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
              'user_declarant_id': 827794},
 'raw_source': {'url': 'https://public-api.nazk.gov.ua/v2/documents/08c243a2-35fe-427d-9b1a-4ce9badaf3ff'},
 'related_entities': {'companies': {'all': ['20064120'],
                                    'owned': [],
                                    'related': ['20064120']},
                      'd

# Process Data

In [7]:
declarations = []
not_2019 = 0
corrected = 0
not_NACP = 0
not_yearly = 0



for i, declaration in enumerate(data):
    if (not declaration['related_entities']['documents']['corrected']):
        if declaration['infocard']['source'] == 'NACP':
            if (declaration['infocard']['document_type'] == "Щорічна"):
                if (declaration['unified_source']['step_0']['declarationYear1'] == "2019"):
                    declarations.append(declaration)
                else: not_2019 += 1
            else: not_yearly += 1
        else: not_NACP += 1
    else: corrected +=1
        

print("Not 2019: ", not_2019, 
      "\nNot Yearly: ", not_yearly, 
      "\nNot NACP: ", not_NACP, 
      "\nCorrected: ", corrected)

print("\nTotal declarations: ", len(data),
      "\nRemained declarations: ", len(declarations))

Not 2019:  0 
Not Yearly:  0 
Not NACP:  0 
Corrected:  23

Total declarations:  787 
Remained declarations:  764


**Розділи декларацій, які нас цікавлять:**
  - 7.Цінні папери
  - 8.Корпоративні права
  - 9.Юридичні особи, трасти або інші подібні правові утворення, кінцевим бенефіціарним власником (контролером) яких є суб’єкт декларування або члени його сім’ї
  
  - 15.Робота за сумісництвом суб’єкта декларування
  - 16.Членство суб’єкта декларування в організаціях та їх органах

In [8]:
# Check how many of the declarations have the entities we are looking for

step_7_securities = 0 
step_8_corp_rights = 0 
step_9_owner_beneficiary = 0 
step_15_part_time_work = 0 
step_16_orgs = 0


for index, r in enumerate(declarations):
    if ("step_7" in r['unified_source'].keys()):
        step_7_securities += 1
    if ("step_8" in r['unified_source'].keys()):
        step_8_corp_rights += 1
    if ("step_9" in r['unified_source'].keys()):
        step_9_owner_beneficiary += 1
    if ("step_15" in r['unified_source'].keys()):
        step_15_part_time_work += 1
    if ("step_16" in r['unified_source'].keys()):
        step_16_orgs += 1
        
print(
    "7.Цінні папери: ", step_7_securities, 
    "\n8.Корпоративні права: ", step_8_corp_rights,
    "\n9.Кінцевий бенефіціарний власник (контролер): ", step_9_owner_beneficiary,
    "\n15.Робота за сумісництвом суб’єкта декларування: ", step_15_part_time_work,
    "\n16.Членство суб’єкта декларування в організаціях та їх органах: ", step_16_orgs
)

7.Цінні папери:  82 
8.Корпоративні права:  247 
9.Кінцевий бенефіціарний власник (контролер):  223 
15.Робота за сумісництвом суб’єкта декларування:  144 
16.Членство суб’єкта декларування в організаціях та їх органах:  338


In [9]:
print(len(declarations))

764


In [122]:
d_with_7 = 0
len7total = 0

for d in declarations:
    if ("step_7" in d['unified_source'].keys()):
        len7 = len(d['unified_source']['step_7'])
        if len7 > 0: # check if there is data in current step
            d_with_7 += 1
            len7total += len7

print(d_with_7, len7total)


d_with_15 = 0
len15total = 0

for d in declarations:
    if ("step_15" in d['unified_source'].keys()):
        len15 = len(d['unified_source']['step_15'])
        if len15 > 0: # check if there is data in current step
            d_with_15 += 1
            len15total += len15

print(d_with_15, len15total)


d_with_16 = 0
len16total = 0

for d in declarations:
    if ("step_16" in d['unified_source'].keys()):
        len16_part_org = len(d['unified_source']['step_16']['part_org'])
        len16_org = len(d['unified_source']['step_16']['org'])

        if len16_part_org > 0 or len16_org > 0: # check if there is data in current step
            d_with_16 += 1
            len16total = len16total + len16_part_org + len16_org

print(d_with_16, len16total)

82 172
144 298
337 990


## Get related companies and Orgs

In [123]:
result_list_7 = []
result_list_8 = []
result_list_9 = []
result_list_15 = []
result_list_16 = []

for index, d in enumerate(declarations):
    
    # some general info about declaration
    current_d_id = d['infocard']['id'], 
    current_d_created_datetime = d['infocard']['created_date']
    current_d_document_type = d['infocard']['document_type'], 
    current_d_year = d['unified_source']['step_0']['declarationYear1'],
    current_d_full_name = d['infocard']['last_name'] + ' ' + d['infocard']['first_name'] + ' ' + d['infocard']['patronymic'],
    current_d_office = d['infocard']['office'], 
    current_d_position = d['infocard']['position']

    
    if ('postType' in d['unified_source']['step_1'].keys()):
        current_d_position_type = d['unified_source']['step_1']['postType']
    else:
        current_d_position_type = np.nan
    
    if ('postCategory' in d['unified_source']['step_1'].keys()):
        current_d_position_category = d['unified_source']['step_1']['postCategory']
    else:
        current_d_position_category = np.nan

    if ('corruptionAffected' in d['unified_source']['step_1'].keys()):
        current_d_corruptionAffected = d['unified_source']['step_1']['corruptionAffected']
    else:
        current_d_corruptionAffected = np.nan
            
    
        
    # step 7 -- Цінні папери
    if ("step_7" in d['unified_source'].keys()):
        if len(d['unified_source']['step_7']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_7'].keys():
                company_ua = np.nan
                company_id = ''
                company_name = ''
                person = ''
                persons= ''

                if ('emitent_ua_company_code' in d['unified_source']['step_7'][c].keys()):
                    if (d['unified_source']['step_7'][c]['emitent_ua_company_code'] != ''):
                            company_ua = 1
                            company_id = d['unified_source']['step_7'][c]['emitent_ua_company_code']
                            company_name = d['unified_source']['step_7'][c]['emitent_ua_company_name']
                if company_id == '':
                    if ('emitent_eng_company_code' in d['unified_source']['step_7'][c].keys()):
                        if (d['unified_source']['step_7'][c]['emitent_eng_company_code'] != ''):
                            company_ua = 0
                            company_id = d['unified_source']['step_7'][c]['emitent_eng_company_code']
                            company_name = d['unified_source']['step_7'][c]['emitent_eng_company_name']

                if ('person' in d['unified_source']['step_7'][c].keys()):
                    person = d['unified_source']['step_7'][c]['person']

                if ('persons' in d['unified_source']['step_7'][c].keys()):
                    persons = d['unified_source']['step_7'][c]['persons']

                result_list_7.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 7, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
                
                
    # step 8 -- Корпоративні права
    if ("step_8" in d['unified_source'].keys()):
        if len(d['unified_source']['step_8']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_8'].keys():
                company_ua = np.nan
                company_id = ''
                company_name = ''
                person = ''
                persons= ''

                if d['unified_source']['step_8'][c]['country'] == '1':
                    company_ua = 1
                else:
                    company_ua = 0

                if ('corporate_rights_company_code' in d['unified_source']['step_8'][c].keys()):
                    company_id = d['unified_source']['step_8'][c]['corporate_rights_company_code']

                if ('name' in d['unified_source']['step_8'][c].keys()):
                    if (d['unified_source']['step_8'][c]['name'] != ''):
                        company_name = d['unified_source']['step_8'][c]['name']
                if company_name == '':
                    if ('en_name' in d['unified_source']['step_8'][c].keys()):
                        if (d['unified_source']['step_8'][c]['en_name'] != ''):
                            company_name = d['unified_source']['step_8'][c]['en_name']

                if ('person' in d['unified_source']['step_8'][c].keys()):
                    person = d['unified_source']['step_8'][c]['person']

                if ('persons' in d['unified_source']['step_8'][c].keys()):
                    persons = d['unified_source']['step_8'][c]['persons']

                result_list_8.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,                                                    
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 8, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
            
            
    
    # step 9 -- Кінцевий бенефіціарний власник (контролер)
    if ("step_9" in d['unified_source'].keys()):
        if len(d['unified_source']['step_9']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_9'].keys():
                company_ua = np.nan
                company_id = ''
                company_name = ''
                person = ''
                persons= ''


                if (d['unified_source']['step_9'][c]['country_beneficial_owner'] != '1'):
                    company_ua = 0
                else:
                    company_ua = 1
                        

                if ('ua_company_code_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                    if (d['unified_source']['step_9'][c]['ua_company_code_beneficial_owner'] != ''):
                            company_id = d['unified_source']['step_9'][c]['ua_company_code_beneficial_owner']
                if company_id == '':
                    if ('company_code_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                        if (d['unified_source']['step_9'][c]['company_code_beneficial_owner'] != ''):
                            company_id = d['unified_source']['step_9'][c]['company_code_beneficial_owner']

  
                if ('company_name_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                    if (d['unified_source']['step_9'][c]['company_name_beneficial_owner'] != ''):
                        company_name = d['unified_source']['step_9'][c]['company_name_beneficial_owner']
                if company_name == '':
                    if ('en_company_name_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                        if (d['unified_source']['step_9'][c]['en_company_name_beneficial_owner'] != ''):
                            company_name = d['unified_source']['step_9'][c]['en_company_name_beneficial_owner']

                if ('person' in d['unified_source']['step_9'][c].keys()):
                    try:
                        person_keys = d['unified_source']['step_9'][c]['person'].keys()
                        if '1' in person_keys:
                            person = 1
                        else:
                            person = list(person_keys)[0]
                    except:
                        print('person does not have keys')
                        person = d['unified_source']['step_9'][c]['person']
                        

                if ('persons' in d['unified_source']['step_9'][c].keys()):
                    persons = d['unified_source']['step_9'][c]['persons']


                result_list_9.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 9, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
            
        
    # step 15 -- Робота за сумісництвом суб’єкта декларування
    if ("step_15" in d['unified_source'].keys()):
        if len(d['unified_source']['step_15']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_15'].keys():
                company_ua = np.nan
                company_id = ''
                company_name = ''
                person = ''
                persons = ''
                description = ''
                emitent_citizen = ''
                paid = ''

                if ('emitent_ua_company_code' in d['unified_source']['step_15'][c].keys()):
                    if (d['unified_source']['step_15'][c]['emitent_ua_company_code'] != ''):
                            company_ua = 1
                            company_id = d['unified_source']['step_15'][c]['emitent_ua_company_code']
                            company_name = d['unified_source']['step_15'][c]['emitent_ua_company_name']
                if company_id == '':
                    if ('emitent_eng_company_code' in d['unified_source']['step_15'][c].keys()):
                        if (d['unified_source']['step_15'][c]['emitent_eng_company_code'] != ''):
                            company_ua = 0
                            company_id = d['unified_source']['step_15'][c]['emitent_eng_company_code']
                            company_name = d['unified_source']['step_15'][c]['emitent_eng_company_name']


                if ('description' in d['unified_source']['step_15'][c].keys()):
                    description = d['unified_source']['step_15'][c]['description']

                if ('emitent_citizen' in d['unified_source']['step_15'][c].keys()):
                    emitent_citizen = d['unified_source']['step_15'][c]['emitent_citizen']

                if ('paid' in d['unified_source']['step_15'][c].keys()):
                    paid = d['unified_source']['step_15'][c]['paid']

                result_list_15.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 15, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name,
                                                    'description': description, 
                                                    'emitent_citizen': emitent_citizen,
                                                    'paid': paid}]))

            
    # step 16 -- Членство суб’єкта декларування в організаціях та їх органах
    if ("step_16" in d['unified_source'].keys()):
        # for orgs
        if len(d['unified_source']['step_16']['org']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_16']['org'].keys():
                org_name = ''
                org_type = ''
                org_id = ''
                org_unit_name = 'org'
                org_unit_type = 'org'

                if ('objectName' in d['unified_source']['step_16']['org'][c].keys()):
                    org_name = d['unified_source']['step_16']['org'][c]['objectName']

                if ('objectType' in d['unified_source']['step_16']['org'][c].keys()):
                    org_type = d['unified_source']['step_16']['org'][c]['objectType']

                if ('reestrCode' in d['unified_source']['step_16']['org'][c].keys()):
                    org_id = d['unified_source']['step_16']['org'][c]['reestrCode']

                result_list_16.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 16, 
                                                    'org_name': org_name,
                                                    'org_type': org_type,
                                                    'org_id': org_id,
                                                    'org_unit_name': org_unit_name,
                                                    'org_unit_type': org_unit_type}]))
            
        # for org parts
        if len(d['unified_source']['step_16']['part_org']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_16']['part_org'].keys():
                org_name = ''
                org_type = ''
                org_id = ''
                org_unit_name = ''
                org_unit_type = ''

                if ('objectName' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_name = d['unified_source']['step_16']['part_org'][c]['objectName']

                if ('objectType' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_type = d['unified_source']['step_16']['part_org'][c]['objectType']

                if ('reestrCode' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_id = d['unified_source']['step_16']['part_org'][c]['reestrCode']  

                if ('unitName' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_unit_name = d['unified_source']['step_16']['part_org'][c]['unitName']   

                if ('unitType' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_unit_type = d['unified_source']['step_16']['part_org'][c]['unitType']

                result_list_16.append(pd.DataFrame([{'id': current_d_id[0],
                                                    'created_datetime': current_d_created_datetime,
                                                    'index': index,
                                                    'document_type': current_d_document_type[0],
                                                    'year': current_d_year[0],
                                                    'full_name': current_d_full_name[0],
                                                    'office': current_d_office[0],
                                                    'position': current_d_position,
                                                    'position_type': current_d_position_type,
                                                    'position_category': current_d_position_category,
                                                    'corruptionAffected': current_d_corruptionAffected,
                                                    'relation_type': 16, 
                                                    'org_name': org_name,
                                                    'org_type': org_type,
                                                    'org_id': org_id,
                                                    'org_unit_name': org_unit_name,
                                                    'org_unit_type': org_unit_type}]))

person does not have keys


In [124]:
print(len(result_list_7),
      len(result_list_8),
      len(result_list_9),
      len(result_list_15),
      len(result_list_16),
)

172 917 2038 298 990


Create summary dataframe for all companies (from steps 7,8,9,15)

In [165]:
companies_df = pd.DataFrame({
    'id': [], 
    'created_datetime': [],
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'position_type': [],
    'position_category': [],
    'corruptionAffected': [],
    'relation_type': [], 
    'person': [], 
    'persons': [], 
    'company_ua': [],
    'company_id': [], 
    'company_name': [],
    'description': [],
    'emitent_citizen': [],
    'paid': []
})                           

In [166]:
companies_df = companies_df.append(result_list_7)
companies_df = companies_df.append(result_list_8)
companies_df = companies_df.append(result_list_9)
companies_df = companies_df.append(result_list_15)

In [167]:
companies_df.head(10)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,corruptionAffected,relation_type,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid
0,nacp_b352acb8-68bc-44b4-a3a4-eebde0e00953,2020-05-29T16:19:20,22.0,Щорічна,2019,Кучер Микола Іванович,Верховна Рада України,народний депутат України,,,Ні,7.0,,j,0.0,1870404,Tansy Holdings Ltd.,NaN,NaN,NaN
0,nacp_b352acb8-68bc-44b4-a3a4-eebde0e00953,2020-05-29T16:19:20,22.0,Щорічна,2019,Кучер Микола Іванович,Верховна Рада України,народний депутат України,,,Ні,7.0,,j,0.0,C71563,Azalea Holdings Ltd.,NaN,NaN,NaN
0,nacp_6003cb68-4147-47f0-8d34-8045fd596c88,2020-06-01T17:34:54,31.0,Щорічна,2019,Драбовський Анатолій Григорович,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,1.0,00373758,"ВАТ ""Вінницький олійно-жировий комбінат""",NaN,NaN,NaN
0,nacp_f041a3d4-0702-41cf-97c4-0b7ebbc8bc0a,2020-05-27T14:22:26,32.0,Щорічна,2019,Третьякова Галина Миколаївна,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,1.0,34486135,"ТОВ ""КУА""ІНВЕСТИЦІЙНИЙ КАПІТАЛ УКРАЇНА""",NaN,NaN,NaN
0,nacp_b51cf2a4-4b0b-411f-a5a8-2399ba6a7c6d,2020-03-31T10:24:19,41.0,Щорічна,2019,Приходько Борис Вікторович,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,1.0,00013480,Міністерство фінансів України,NaN,NaN,NaN
0,nacp_b51cf2a4-4b0b-411f-a5a8-2399ba6a7c6d,2020-03-31T10:24:19,41.0,Щорічна,2019,Приходько Борис Вікторович,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,1.0,00013480,Міністерство фінансів України,NaN,NaN,NaN
0,nacp_6dc96cd6-38d9-482d-ad29-0cd4feaf7d67,2020-03-31T18:49:16,45.0,Щорічна,2019,Циба Тетяна Вікторівна,Верховна Рада України,народний депутат України,Посадова особа юридичної особи публічного права,,Так,7.0,,,1.0,24432974,арселор мітал кривий ріг,NaN,NaN,NaN
0,nacp_c26141d3-3d66-4b69-bdbc-fdb97269a676,2020-05-25T11:23:47,53.0,Щорічна,2019,Рахманін Сергій Іванович,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,1.0,40512094,Товариство з обмеженою відповідальністю СТАТУС...,NaN,NaN,NaN
0,nacp_e1b5825a-378d-4c2b-ae8f-72e190100617,2020-05-31T19:58:37,55.0,Щорічна,2019,Льовочкіна Юлія Володимирівна,Верховна Рада України,Народний депутат України,,,Ні,7.0,,j,1.0,41679371,Корпоративний інвестиційний фонд Акціонерне то...,NaN,NaN,NaN
0,nacp_82c76231-5ae5-4a17-bc8b-33a7865c82cc,2020-05-31T20:26:03,56.0,Щорічна,2019,Арахамія Давид Георгійович,Верховна Рада України,Народний депутат України,,,Ні,7.0,,,0.0,202281,Global Bay Venture Limited,NaN,NaN,NaN


In [168]:
companies_df.groupby(['office']).count()

,id,created_datetime,index,document_type,year,full_name,position,position_type,position_category,corruptionAffected,relation_type,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid
office,,,,,,,,,,,,,,,,,,,
Апарат Верховної Ради України,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,3,3,3
Апарат Верховної Ради України,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1
"Апарат Верховної Ради України, Херсонська обласна рада",4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0
Апарат Верховної Ради України/Пологівська міська рада Запорізької області,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
ВЕРХОВНА РАДА УКРАЇНИ,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,124,2,2,2
Верховна Рада,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,0,0,0
Верховна Рада Україна,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0
Верховна Рада України,2882,2882,2882,2882,2882,2882,2882,2882,2882,2882,2882,2882,2882,2851,2882,2882,245,245,245
Верховна Рада України,154,154,154,154,154,154,154,154,154,154,154,154,154,152,154,154,23,23,23


Create summary dataframe for all organizations (from step 16)

In [169]:
organizations_df = pd.DataFrame({
    'id': [],
    'created_datetime': [],
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'position_type': [],
    'position_category': [],
    'corruptionAffected': [],
    'relation_type': [], 
    'org_name': [], 
    'org_type': [], 
    'org_id': [],
    'org_unit_name': [], 
    'org_unit_type': []
})   

In [170]:
organizations_df = organizations_df.append(result_list_16)
organizations_df

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,corruptionAffected,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type
0,nacp_3a9e9b55-3621-4436-9fd6-319c861a4624,2020-05-27T16:54:50,1.0,Щорічна,2019,Вельможний Сергій Анатолійович,народний депутат України,народний депутат України,Посада державної служби,А,Так,16.0,Національна асоціація адвокатів України,Саморегулівне чи самоврядне професійне об’єднання,38488439,org,org
0,nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,2020-05-25T01:08:30,15.0,Щорічна,2019,Лунченко Валерій Валерійович,Верховна Рада України,Народний депутат України,,,Ні,16.0,Благодійна організація благодійний фонд Валері...,Благодійна організація,38629273,org,org
0,nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,2020-05-25T01:08:30,15.0,Щорічна,2019,Лунченко Валерій Валерійович,Верховна Рада України,Народний депутат України,,,Ні,16.0,"Громадська організація ""Фундація розвитку зака...",Громадське об’єднання,40896515,org,org
0,nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,2020-05-25T01:08:30,15.0,Щорічна,2019,Лунченко Валерій Валерійович,Верховна Рада України,Народний депутат України,,,Ні,16.0,"ГРОМАДСЬКА ОРГАНІЗАЦІЯ ""ВСЕУКРАЇНСЬКА АСОЦІАЦІ...",Громадське об’єднання,41092302,org,org
0,nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,2020-05-25T01:08:30,15.0,Щорічна,2019,Лунченко Валерій Валерійович,Верховна Рада України,Народний депутат України,,,Ні,16.0,Благодійна організація благодійний фонд Валері...,Благодійна організація,38629273,Засновник,Керівні органи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,nacp_5ded6455-0200-46ea-b365-ea7aa392b170,2020-05-26T14:32:45,756.0,Щорічна,2019,Луцков Володимир Ігорович,"Апарат Верховної ради України, Товариство з об...",Помічник консультант на громадських засадах на...,Посадова особа юридичної особи публічного права,,Ні,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,Голова правління,Керівні органи
0,nacp_12a6d842-3258-444a-953e-cc576f726b7a,2020-05-23T15:09:48,757.0,Щорічна,2019,Луцков Володимир Ігорович,"Апарат Верховної ради України, Товариство з об...",Помічник консультант на громадських засадах на...,Посадова особа юридичної особи публічного права,,Ні,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,org,org
0,nacp_12a6d842-3258-444a-953e-cc576f726b7a,2020-05-23T15:09:48,757.0,Щорічна,2019,Луцков Володимир Ігорович,"Апарат Верховної ради України, Товариство з об...",Помічник консультант на громадських засадах на...,Посадова особа юридичної особи публічного права,,Ні,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,Голова правління,Керівні органи
0,nacp_e9105bf5-1457-438d-8c63-4626c6dab6c5,2020-03-31T20:30:46,758.0,Щорічна,2019,Тимченко Тетяна Юріївна,"Міністерство юстиції України, Департамент держ...",начальник відділу розгляду запитів і звернень ...,Посада державної служби,Б,Ні,16.0,Первинна профспілкова організація Міністерства...,Громадське об’єднання,42171490,org,org


In [171]:
# convert datetime data to datetime format
organizations_df['created_datetime'] = pd.to_datetime(organizations_df['created_datetime'])
companies_df['created_datetime'] = pd.to_datetime(companies_df['created_datetime'])

In [172]:
#companies_df.to_csv('companies_df.csv', index=False)
#organizations_df.to_csv('organizations_df.csv', index=False)

## Process data - verify declaration uniqueness, create drop lists, check company codes, create complanie dict and save the data sets

### Remove declarations with diferent ids but from the same people

In [173]:
companies_df['full_name_office_position'] = companies_df.full_name  + ' ' + companies_df.office + ' ' + companies_df.position
companies_df['full_name_office_position'] = companies_df['full_name_office_position'].str.lower()


organizations_df['full_name_office_position'] = organizations_df.full_name  + ' ' + organizations_df.office + ' ' + organizations_df.position
organizations_df['full_name_office_position'] = organizations_df['full_name_office_position'].str.lower()


companies_df['full_name_office_position']

0    кучер микола іванович верховна рада україни на...
0    кучер микола іванович верховна рада україни на...
0    драбовський анатолій григорович верховна рада ...
0    третьякова галина миколаївна верховна рада укр...
0    приходько борис вікторович верховна рада украї...
                           ...                        
0    вагнєр вікторія олександрівна верховна рада ук...
0    колесников вячеслав ігорович апарат верховної ...
0    колесников вячеслав ігорович апарат верховної ...
0    бакумов олександр сергійович верховна рада укр...
0    насінник сергій валерійович управління справам...
Name: full_name_office_position, Length: 3425, dtype: object

In [174]:
comp_people_map = companies_df.groupby(["full_name_office_position"])["id"].nunique().reset_index(name="unique_guids")
print('companies')
print(len(comp_people_map[comp_people_map.unique_guids > 1]))

print('organizations')
org_people_map = organizations_df.groupby(["full_name_office_position"])["id"].nunique().reset_index(name="unique_guids")
print(len(org_people_map[org_people_map.unique_guids > 1]))


companies
62
organizations
51


In [175]:
print('companies')
print(len(companies_df.full_name), len(set(companies_df.full_name)))
print(len(companies_df.id), len(set(companies_df.id)))
print(len(comp_people_map.full_name_office_position), len(set(comp_people_map.full_name_office_position)))

print('organizations')
print(len(organizations_df.full_name), len(set(organizations_df.full_name)))
print(len(organizations_df.id), len(set(organizations_df.id)))
print(len(org_people_map.full_name_office_position), len(set(org_people_map.full_name_office_position)))

companies
3425 283
3425 353
283 283
organizations
990 275
990 337
276 276


In [176]:
companies_df = companies_df.merge(comp_people_map, how='left', on='full_name_office_position')
companies_df.head(1)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid,full_name_office_position,unique_guids
0,nacp_b352acb8-68bc-44b4-a3a4-eebde0e00953,2020-05-29 16:19:20,22.0,Щорічна,2019,Кучер Микола Іванович,Верховна Рада України,народний депутат України,,,...,,j,0.0,1870404,Tansy Holdings Ltd.,NaN,NaN,NaN,кучер микола іванович верховна рада україни на...,1


In [157]:
unique_naming_companies_df = companies_df.sort_values('created_datetime').drop_duplicates(['full_name_office_position', 
                                                                                           'company_id'], 
                                                                                          keep='last')
print('companies_df')
print(len(companies_df.full_name), len(set(companies_df.full_name)))
print(len(companies_df.id), len(set(companies_df.id)))

print('unique_naming_companies_df')
print(len(unique_naming_companies_df.full_name), len(set(unique_naming_companies_df.full_name)))
print(len(unique_naming_companies_df.id), len(set(unique_naming_companies_df.id)))


companies_df
3425 283
3425 353
unique_naming_companies_df
1711 283
1711 285


In [177]:
organizations_df = organizations_df.merge(org_people_map, how='left', on='full_name_office_position')
organizations_df.head(1)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,corruptionAffected,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type,full_name_office_position,unique_guids
0,nacp_3a9e9b55-3621-4436-9fd6-319c861a4624,2020-05-27 16:54:50,1.0,Щорічна,2019,Вельможний Сергій Анатолійович,народний депутат України,народний депутат України,Посада державної служби,А,Так,16.0,Національна асоціація адвокатів України,Саморегулівне чи самоврядне професійне об’єднання,38488439,org,org,вельможний сергій анатолійович народний депута...,1


In [178]:
unique_naming_organizations_df = organizations_df.sort_values('created_datetime').drop_duplicates(['full_name_office_position', 
                                                                                           'org_id'], 
                                                                                          keep='last')
print('organizations_df')
print(len(organizations_df.full_name), len(set(organizations_df.full_name)))
print(len(organizations_df.id), len(set(organizations_df.id)))

print('unique_naming_organizations_df')
print(len(unique_naming_organizations_df.full_name), len(set(unique_naming_organizations_df.full_name)))
print(len(unique_naming_organizations_df.id), len(set(unique_naming_organizations_df.id)))


organizations_df
990 275
990 337
unique_naming_organizations_df
576 275
576 276


In [156]:
unique_naming_companies_df.to_csv('no_dupl_companies_df.csv', index=False)
companies_df.to_csv('with_dupl_companies_df.csv', index=False)

unique_naming_organizations_df.to_csv('no_dupl_organizations_df.csv', index=False)
organizations_df.to_csv('with_dupl_organizations_df.csv', index=False)

In [179]:
companies_df = unique_naming_companies_df
organizations_df = unique_naming_organizations_df

### Create drop lists -- lists of companies that appear just for a single time 
to use it later for filtering out them from the graphs.

See how many folks have a single related company

In [180]:
by_decl_id = companies_df.groupby(["id"])["company_id"].count().reset_index(name="count_companies")
by_decl_id.head(5)

,id,count_companies
0,nacp_001d1aa8-b060-458a-91f7-b10f832dc1b1,4
1,nacp_004a8f7f-d7bc-4f13-a90a-2f6ba528b6bd,1
2,nacp_00ac8bd5-96b4-4983-a09f-19e976cd2a03,7
3,nacp_00dabaef-3780-4e08-b327-cbdc59fc02c4,1
4,nacp_022001af-2b86-463f-a442-667f0731b65c,4


In [181]:
by_decl_id.groupby(["count_companies"])["id"].count().reset_index(name="count_users")

,count_companies,count_users
0,1,115
1,2,48
2,3,29
3,4,27
4,5,13
5,6,6
6,7,7
7,8,3
8,9,4
9,10,3


There are 115 users who have just a single company. Create a list of the for filter them out later on.

In [182]:
by_decl_id.loc[by_decl_id['count_companies'] == 1]

,id,count_companies
1,nacp_004a8f7f-d7bc-4f13-a90a-2f6ba528b6bd,1
3,nacp_00dabaef-3780-4e08-b327-cbdc59fc02c4,1
6,nacp_05992233-bb24-48de-a1dd-5d4037e70560,1
11,nacp_07f9eab1-42c4-4823-8568-596315c5a1f3,1
14,nacp_0d4ff3f0-2254-400b-9abf-58c497354ee0,1
...,...,...
271,nacp_f6841618-639e-4fa6-9b87-edc61c3ebf59,1
275,nacp_fab09b3c-bd78-4ce5-bf49-4abfa862d3e2,1
276,nacp_fb06fb32-019f-4286-a6fe-97b45a24a428,1
283,nacp_ff88a131-2f9c-49e0-bad4-db2c72ef91d6,1


In [183]:
by_company_id = companies_df.groupby(["company_id"])["id"].count().reset_index(name="count_decl")
by_company_id.loc[by_company_id['count_decl'] == 1]

,company_id,count_decl
1,00000000,1
2,0000294427,1
3,0000361902,1
4,0000426113,1
5,0000546819,1
...,...,...
1658,HRA 12010,1
1659,HRA 3128,1
1660,HRB 101996,1
1661,HRB 95356,1


In [184]:
folks_with_single_company = list(by_decl_id.loc[by_decl_id['count_companies'] == 1]['id'])
companies_appear_single_time = list(by_company_id.loc[by_company_id['count_decl'] == 1]['company_id'])

### Check the organization codes and correct them

In [185]:
organizations_df.head(2)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,corruptionAffected,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type,full_name_office_position,unique_guids
938,nacp_0bc6dacd-c8ce-4d63-a9c7-98c7cdcac81d,2020-01-04 14:41:19,691.0,Щорічна,2019,Богомазов Павло Сергійович,Верховна Рада України,ПОМІЧНИК-КОНСУЛЬТАНТ НАРОДНОГО ДЕПУТАТА УКРАЇНИ,,,Ні,16.0,ПОЛІТИЧНА ПАРТІЯ РУХ НОВИХ СИЛ МИХАЙЛА СААКАШВІЛІ,Громадське об’єднання,39634331,org,org,богомазов павло сергійович верховна рада украї...,1
845,nacp_35cd7c41-c181-441d-9009-2688c68c25db,2020-01-04 23:53:30,556.0,Щорічна,2019,Кіхтан Оксана Ярославівна,Верховна Рада України,помічник - консультант Народного депутата Укра...,,,Ні,16.0,ПОЛІТИЧНА ПАРТІЯ РУХ НОВИХ СИЛ МИХАЙЛА СААКАШВІЛІ,Громадське об’єднання,39634331,org,org,кіхтан оксана ярославівна верховна рада україн...,1


In [186]:
# check if we have empty company IDs or names
print('total records: ', len(organizations_df))

empty_ids = organizations_df.loc[(organizations_df['org_id'] == '')] 
print('\nrecords with empty ids: ', len(empty_ids))
print('empty ids share: ', len(empty_ids)/len(organizations_df))

empty_names = organizations_df.loc[(organizations_df['org_name'] == '')] 
print('\nrecords with empty names: ', len(empty_names))
print('empty names share: ', len(empty_names)/len(organizations_df))


empty_names_ids = organizations_df.loc[((organizations_df['org_name'] == '')) & (organizations_df['org_id'] == '')] 
print('\nrecords with both empty names and empty IDs: ', len(empty_names_ids))
print('both empty names and empty IDs share: ', len(empty_names_ids)/len(organizations_df))


total records:  576

records with empty ids:  0
empty ids share:  0.0

records with empty names:  0
empty names share:  0.0

records with both empty names and empty IDs:  0
both empty names and empty IDs share:  0.0


In [187]:
clean_organizations_df = organizations_df.loc[organizations_df['org_id'] != ''].copy(deep=True)
len(clean_organizations_df)

576

In [188]:
print('orgs with correct EDRPOU length: ', 
      len(clean_organizations_df[(clean_organizations_df['org_id'].str.len() == 8)]))

print('orgs with non-8-char EDRPOU length: ', 
      len(clean_organizations_df[(clean_organizations_df['org_id'].str.len() != 8)]))

orgs with correct EDRPOU length:  574
orgs with non-8-char EDRPOU length:  2


In [189]:
# We won't delete orgs from here because we do not have country filter

### Check the company codes and correct them

In [190]:
# check if we have empty company IDs or names
print('total records: ', len(companies_df))

empty_ids = companies_df.loc[(companies_df['company_id'] == '')] 
print('\nrecords with empty ids: ', len(empty_ids))
print('empty ids share: ', len(empty_ids)/len(companies_df))

empty_names = companies_df.loc[(companies_df['company_name'] == '')] 
print('\nrecords with empty names: ', len(empty_names))
print('empty names share: ', len(empty_names)/len(companies_df))


empty_names_ids = companies_df.loc[((companies_df['company_name'] == '')) & (companies_df['company_id'] == '')] 
print('\nrecords with both empty names and empty IDs: ', len(empty_names_ids))
print('both empty names and empty IDs share: ', len(empty_names_ids)/len(companies_df))


total records:  1711

records with empty ids:  20
empty ids share:  0.011689070718877849

records with empty names:  16
empty names share:  0.009351256575102279

records with both empty names and empty IDs:  16
both empty names and empty IDs share:  0.009351256575102279


In [191]:
clean_companies_df = companies_df.loc[companies_df['company_id'] != ''].copy(deep=True)
len(clean_companies_df)

1691

In [192]:
print('non-ukraine companies', len(clean_companies_df[(clean_companies_df['company_ua'] != 1)]))
print('ukraine companies', len(clean_companies_df[(clean_companies_df['company_ua'] == 1)]))
print('ukraine companies with correct EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() == 8)]))
print('ukraine companies with too long EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() > 8)]))
print('ukraine companies with too short EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() < 8)]))

non-ukraine companies 349
ukraine companies 1342
ukraine companies with correct EDRPOU length 1331
ukraine companies with too long EDRPOU length 11
ukraine companies with too short EDRPOU length 0


In [193]:
# Drop all entries where Ukraine's companu EDRPOU code is not correct (either too short or too long)
                        
clean_companies_df = clean_companies_df[((clean_companies_df['company_ua'] == 1) & 
                                           (clean_companies_df['company_id'].str.len() == 8)) | 
                                          (clean_companies_df['company_ua'] == 0)]


len(clean_companies_df)

1680

In [194]:
# Process edrpou code 
# Generally we delete all cases where length of EDRPOU code is < 4
# For Ukrainian companies - so that the length is 8 characters, if it is less, we add zeros before 


#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 4)), 'company_id_r'] = '0000' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 4))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 5)), 'company_id_r'] = '000' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 5))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 6)), 'company_id_r'] = '00' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 6))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 7)), 'company_id_r'] = '0' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 7))]['company_id']

#p[p['company_ua'] == 1]['company_id_r'].str.len().value_counts()

#p['company_id_r'].str.len().value_counts()

In [195]:
clean_companies_df.head(3)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid,full_name_office_position,unique_guids
3411,nacp_0bc6dacd-c8ce-4d63-a9c7-98c7cdcac81d,2020-01-04 14:41:19,691.0,Щорічна,2019,Богомазов Павло Сергійович,Верховна Рада України,ПОМІЧНИК-КОНСУЛЬТАНТ НАРОДНОГО ДЕПУТАТА УКРАЇНИ,,,...,,,1.0,41000562,"громадська організація ""Рух нових сил Михайла ...",голова,"Юридична особа, зареєстрована в Україні",Не оплачувана,богомазов павло сергійович верховна рада украї...,1
3412,nacp_0bc6dacd-c8ce-4d63-a9c7-98c7cdcac81d,2020-01-04 14:41:19,691.0,Щорічна,2019,Богомазов Павло Сергійович,Верховна Рада України,ПОМІЧНИК-КОНСУЛЬТАНТ НАРОДНОГО ДЕПУТАТА УКРАЇНИ,,,...,,,1.0,40528675,"громадська організація ""Національна антикорупц...",голова,"Юридична особа, зареєстрована в Україні",Не оплачувана,богомазов павло сергійович верховна рада украї...,1
3413,nacp_0bc6dacd-c8ce-4d63-a9c7-98c7cdcac81d,2020-01-04 14:41:19,691.0,Щорічна,2019,Богомазов Павло Сергійович,Верховна Рада України,ПОМІЧНИК-КОНСУЛЬТАНТ НАРОДНОГО ДЕПУТАТА УКРАЇНИ,,,...,,,1.0,42069557,"адвокатське об'єднання ""БОГОМАЗОВ ТА ПАРТНЕРИ""",керуючий партнер,"Юридична особа, зареєстрована в Україні",Не оплачувана,богомазов павло сергійович верховна рада украї...,1


### Create data sets of pairs,  company and org dicts

In [196]:
id_company_pairs = clean_companies_df.drop_duplicates(subset = ['id','company_id']) #['id','company_id','relation_type'])
print(len(clean_companies_df), len(id_company_pairs))

1680 1680


In [197]:
name_company_pairs = clean_companies_df.drop_duplicates(subset = ['full_name','company_id']) #['id','company_id','relation_type'])
print(len(clean_companies_df), len(name_company_pairs))

1680 1680


In [198]:
id_org_pairs = clean_organizations_df.drop_duplicates(subset = ['id','org_id']) #['id','company_id','relation_type'])
print(len(clean_organizations_df), len(id_org_pairs))

576 576


In [199]:
# Create a mapper for finding comapny name by company code
# Use the most popular comapny name 

company_map = clean_companies_df[['company_id', "company_name", "company_ua"]]
company_map = company_map.groupby(['company_id', "company_name", "company_ua"]).size().reset_index(name='counts')

comp_ids_and_counts = company_map.groupby(['company_id'])['counts'].agg('sum')

company_map = company_map.sort_values('counts', ascending=False).groupby('company_id').head(1)
company_map.drop('counts', axis=1,inplace=True)

company_map = company_map.join(comp_ids_and_counts, how='inner', on='company_id')

company_map

,company_id,company_name,company_ua,counts
70,02070944,Київський національний університет імені Тарас...,1.0,3
314,22907278,Інститут законодавства Верховної Ради України,1.0,2
1252,40632877,КНИГОЛАВ,1.0,2
164,141488,NOVEMBE VENTURES LIMITED,0.0,2
69,02070938,Національний університет харчових технологій,1.0,2
...,...,...,...,...
558,32285073,КОНТУР,1.0,1
557,32285068,ДЕБЮТ,1.0,1
556,32285052,ПРИВАТНЕ ПІДПРИЄМСТВО «ЛІДЕР»,1.0,1
555,32280489,ДНІПРОВСЬКИЙ ТЕПЛИЧНИЙ КОМБІНАТ,1.0,1


In [200]:
# Create a mapper for finding organization name by company code
# Use the most popular organization name 

org_map = clean_organizations_df[['org_id', "org_name"]]
org_map = org_map.groupby(['org_id', "org_name"]).size().reset_index(name='counts')

org_ids_and_counts = org_map.groupby(['org_id'])['counts'].agg('sum')

org_map = org_map.sort_values('counts', ascending=False).groupby('org_id').head(1)
org_map.drop('counts', axis=1,inplace=True)

org_map = org_map.join(org_ids_and_counts, how='inner', on='org_id')

org_map

,org_id,org_name,counts
267,38488439,Національна асоціація адвокатів України,30
529,43194256,Громадська організація Я українець - мені не в...,3
534,43297484,Благодійний фонд Я українець - мені не все одно,3
518,42654646,Громадська організація Олексія Леонова Я Одеси...,3
406,40224612,ПРОФЕШЕНАЛ ГАВЕРМЕНТ,3
...,...,...,...
175,35402103,"ГО садівницьке товариство ""Пролісок""",1
174,35361193,Федерація хокею Хмельницької області,1
173,35305233,Асоціація випускників Хмельницького університе...,1
172,35297432,АСОЦІАЦІЯ СПОРТИВНОЇ БОРОТЬБИ ВІННИЦЬКОЇ ОБЛАСТІ,1


### Save the data sets into .csv files

In [201]:
clean_companies_df.to_csv('company_clean_df.csv', index=False)
id_company_pairs.to_csv('company_id_pairs.csv', index=False)
company_map.to_csv('company_map.csv', index=False)


clean_organizations_df.to_csv('org_clean_df.csv', index=False)
id_org_pairs.to_csv('org_id_pairs.csv', index=False)
org_map.to_csv('org_map.csv', index=False)

# Building Graphs

## Company Graphs

In [202]:
import networkx as nx
from networkx.algorithms import community 

import matplotlib.pyplot as plt
from pyvis.network import Network

In [203]:
f_comp = id_company_pairs.merge(company_map[['company_id', "company_name", "counts"]], how='left', on='company_id')

In [204]:
f_comp.head(1)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,company_ua,company_id,company_name_x,description,emitent_citizen,paid,full_name_office_position,unique_guids,company_name_y,counts
0,nacp_0bc6dacd-c8ce-4d63-a9c7-98c7cdcac81d,2020-01-04 14:41:19,691.0,Щорічна,2019,Богомазов Павло Сергійович,Верховна Рада України,ПОМІЧНИК-КОНСУЛЬТАНТ НАРОДНОГО ДЕПУТАТА УКРАЇНИ,,,...,1.0,41000562,"громадська організація ""Рух нових сил Михайла ...",голова,"Юридична особа, зареєстрована в Україні",Не оплачувана,богомазов павло сергійович верховна рада украї...,1,"громадська організація ""Рух нових сил Михайла ...",1


In [88]:
# Create data for graph

subset_comp_ids = f_comp[['id','company_id']]
subset_comp_names = f_comp[['id','company_name_y']]
subset_names_names = f_comp[['full_name','company_name_y']]

print(len(subset_comp_ids), len(subset_comp_names), len(subset_names_names))


2364 2364 2364


In [205]:
mask_muli_owners = (f_comp['counts'] > 1)

In [208]:
subset_multi_names_names = f_comp[mask_muli_owners][['full_name','company_name_y']]
#subset_multi_names_names = f_comp[mask_muli_owners][['id','company_id']]

print(len(subset_multi_names_names))

50


In [72]:
tuples = [tuple(x) for x in subset_names_names.values]

G_comp = nx.Graph(tuples)
print(nx.info(G_comp))

Name: 
Type: Graph
Number of nodes: 1917
Number of edges: 1669
Average degree:   1.7413


In [73]:
density = nx.density(G_comp)
print("Network density:", density)

Network density: 0.0009088008294100799


In [209]:
nt = Network(notebook=True)
nt.from_nx(G_comp)
nt.show_buttons(filter_=['physics'])
nt.show('G_comp.html')

In [210]:
tuples_multi = [tuple(x) for x in subset_multi_names_names.values]

G_comp_multi = nx.Graph(tuples_multi)
print(nx.info(G_comp_multi))

nt_multi = Network(notebook=True)
nt_multi.from_nx(G_comp_multi)
nt_multi.show_buttons(filter_=['physics'])
nt_multi.show('G_comp_multi.html')

Name: 
Type: Graph
Number of nodes: 66
Number of edges: 50
Average degree:   1.5152
